In [8]:
# Setup  Chunk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
import time

# NLTK Stop words extended
stop_words = stopwords.words('english')
lemma = WordNetLemmatizer()


import spacy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import re
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# import numpy as np



In [3]:
df = pd.read_csv('Donald-Tweets!.csv')

In [4]:
df.head()

,Date,Time,Tweet_Text,Type,Media_Type,Hashtags,Tweet_Id,Tweet_Url,twt_favourites_IS_THIS_LIKE_QUESTION_MARK,Retweets,Unnamed: 10,Unnamed: 11
0,16-11-11,15:26:37,Today we express our deepest gratitude to all ...,text,photo,ThankAVet,7.970000e+17,https://twitter.com/realDonaldTrump/status/797...,127213,41112,NaN,NaN
1,16-11-11,13:33:35,Busy day planned in New York. Will soon be mak...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/797...,141527,28654,NaN,NaN
2,16-11-11,11:14:20,Love the fact that the small groups of protest...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/797...,183729,50039,NaN,NaN
3,16-11-11,2:19:44,Just had a very open and successful presidenti...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/796...,214001,67010,NaN,NaN
4,16-11-11,2:10:46,A fantastic day in D.C. Met with President Oba...,text,NaN,NaN,7.970000e+17,https://twitter.com/realDonaldTrump/status/796...,178499,36688,NaN,NaN


In [5]:
df = df[['Tweet_Text','Retweets']]

In [6]:
df.shape

(7375, 2)

In [9]:
# Function to clean text
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

In [12]:
corpus = process_words(df['Tweet_Text'])  # corpus cleaned of html tags, puncs, lemmas
corpus = [[word for word in sublist if len(word)>2] for sublist in corpus] # remove words with less than 2 characters
corpus = [' '.join(i) for i in corpus]

In [13]:
df['Cleaned Tweet'] = corpus
df['Cleaned Tweet'].replace('', np.nan, inplace=True)
df.dropna(subset=['Cleaned Tweet'], inplace=True)

In [25]:
# import module for train-test split
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(df['Cleaned Tweet'], df['Retweets'],test_size=0.3,random_state=42)


In [26]:
tfidf = TfidfVectorizer(ngram_range=(1,3))
tfidf_vect = tfidf.fit(Xtrain)


In [27]:
xtrain_tfidf =  tfidf_vect.transform(Xtrain)
xtest_tfidf =  tfidf_vect.transform(Xtest)

In [28]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(xtrain_tfidf,ytrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [29]:
#To retrieve the intercept:
print(lr.intercept_)
#For retrieving the slope:
print(lr.coef_)

3873.1537046865496
[-445.50198391 4103.34241916 -531.98461389 ...  137.57925374  137.57925374
 1713.20298369]


In [30]:
y_pred = lr.predict(xtest_tfidf)


In [31]:
results = pd.DataFrame({'Actual': ytest, 'Predicted': y_pred})
results

,Actual,Predicted
4119,992,12591.991501
1364,7527,-1126.734921
4915,1396,1395.058605
2943,3394,3873.153705
432,8959,2116.600734
...,...,...
2706,3197,-1196.274241
6682,523,-3270.928181
1883,4735,-10.375150
3936,884,-901.866752


In [32]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(ytest, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, y_pred)))

Mean Absolute Error: 3690.1659073561786
Mean Squared Error: 33114230.68849303
Root Mean Squared Error: 5754.4965625581035
